In [1]:
from src.config.path_config import *
import os
import glob
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

c:\Users\승범 pc\Desktop\study\deeplearning\naver_boot\level1-semantictextsimilarity-nlp-07\project1_virtual\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
def load_csv_data( path, column):
    csv_data = []
    csv_files = glob.glob(os.path.join(path, '*.csv'))
    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        csv_data.append(df[column].values)
    return len(csv_files), torch.tensor(np.array(csv_data).T, dtype=torch.float32) # Shape: (num_samples, num_csvs)

In [12]:
# 평균 추론
_, mean_data = load_csv_data(ENSEMBLE_OUTPUT, column='target')
mean_data = torch.mean(mean_data, dim=1, keepdim=True)

In [13]:
mean_data = np.array(mean_data)

In [14]:
output = pd.read_csv(OUTPUT_PATH)
output['target'] = mean_data
output_filename = f'Ensemble_mean.csv'
output.to_csv(output_filename, index=False)

In [15]:
target_data = pd.read_csv('./results/train/train_50_electra_base.csv')
target_data = target_data['label']
target_data = torch.tensor(target_data, dtype=torch.float32)
len(target_data)

550

In [16]:
num_csvs, linear_data = load_csv_data(ENSEMBLE_TRAIN, column='target')

In [17]:
model =  nn.Linear(num_csvs, 1)
num_epochs=100
lr=0.001
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

for epoch in tqdm(range(num_epochs)):
    optimizer.zero_grad()
    outputs = model(linear_data)
    loss = criterion(outputs, target_data)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

  0%|          | 0/100 [00:00<?, ?it/s]c:\Users\승범 pc\Desktop\study\deeplearning\naver_boot\level1-semantictextsimilarity-nlp-07\project1_virtual\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([550])) that is different to the input size (torch.Size([550, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 100/100 [00:00<00:00, 1427.18it/s]

Epoch [10/100], Loss: 9.2853
Epoch [20/100], Loss: 8.6732
Epoch [30/100], Loss: 8.1020
Epoch [40/100], Loss: 7.5733
Epoch [50/100], Loss: 7.0869
Epoch [60/100], Loss: 6.6416
Epoch [70/100], Loss: 6.2356
Epoch [80/100], Loss: 5.8667
Epoch [90/100], Loss: 5.5328
Epoch [100/100], Loss: 5.2316


In [18]:
model.eval()
_, mean_data = load_csv_data(ENSEMBLE_OUTPUT, column='target')

In [19]:
predictions = model(mean_data)

In [36]:
output = pd.read_csv(OUTPUT_PATH)
output['target'] = predictions
output_filename = f'Ensemble_linear.csv'
output.to_csv(output_filename, index=False)

In [47]:
mean_data, predictions

(tensor([[2.9863, 3.2383, 2.4691, 3.7734],
         [4.4141, 3.9688, 4.3391, 4.2198],
         [1.4971, 1.2686, 1.5906, 1.9185],
         ...,
         [3.6953, 3.7500, 3.8851, 3.9554],
         [3.1133, 3.8379, 3.4891, 3.9165],
         [4.7734, 4.9062, 4.9679, 5.3005]]),
 tensor([[2.4123],
         [3.3526],
         [1.5244],
         ...,
         [3.0198],
         [2.7745],
         [3.7742]]))